# Загружаем библиотеки и датафрейм
---

In [2]:
# Устанавливаем в среду пакет surprise 
%pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195811 sha256=024346cabcc217b5538df5f3c233bf4f523a976a08eda3d28f655bc52145def4
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
from surprise import SVD


In [2]:
# загружаем исходный файл
data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [3]:
work_data = data.copy()

In [4]:
work_data.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [5]:
work_data['rating'] = work_data['event_type'].map({'show_vacancy': 0,
                                     'preview_click_vacancy': 0,
                                     'click_favorite': 0,
                                     'preview_click_favorite': 0,
                                     'click_response': 1,
                                     'preview_click_response': 1,
                                     'click_contacts': 1,
                                     'preview_click_contacts': 1,
                                     'click_phone': 1,
                                     'preview_click_phone': 1})

In [21]:
work_data['rating'] = work_data['event_type'].map({'show_vacancy': 1,
                                     'preview_click_vacancy': 2,
                                     'click_favorite': 3,
                                     'preview_click_favorite': 4,
                                     'click_response': 5,
                                     'preview_click_response': 6,
                                     'click_contacts': 7,
                                     'preview_click_contacts': 8,
                                     'click_phone': 9,
                                     'preview_click_phone': 10})

In [6]:
df_ratings = work_data[['cookie_id', 'vacancy_id_', 'rating']]

In [7]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   cookie_id    object
 1   vacancy_id_  int64 
 2   rating       int64 
dtypes: int64(2), object(1)
memory usage: 281.4+ MB


In [8]:
df_ratings = df_ratings.groupby(['cookie_id','vacancy_id_'])['rating'].max().reset_index()

In [9]:
df_ratings.head()

,cookie_id,vacancy_id_,rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0


In [10]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678585 entries, 0 to 4678584
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   cookie_id    object
 1   vacancy_id_  int64 
 2   rating       int64 
dtypes: int64(2), object(1)
memory usage: 107.1+ MB


In [11]:
mask = df_ratings['rating'] == 1

In [12]:
df_ratings.loc[mask][['cookie_id','vacancy_id_', 'rating']]

,cookie_id,vacancy_id_,rating
18,0000e63f423542fe8090575b611cea4d,122287,1
20,0000e63f423542fe8090575b611cea4d,137592,1
21,0000e63f423542fe8090575b611cea4d,141592,1
22,0000e63f423542fe8090575b611cea4d,158280,1
25,0000e63f423542fe8090575b611cea4d,251362,1
...,...,...,...
4678580,ffffdb17f45b4032b386d691d52e6c00,107125,1
4678581,ffffdb17f45b4032b386d691d52e6c00,107893,1
4678582,ffffdb17f45b4032b386d691d52e6c00,150843,1
4678583,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [13]:
# перемешаем рейтиги
df_ratings = df_ratings.sample(frac=1)

In [14]:
df_final = df_ratings

Возьмем за основу наш базовый датасет, он учится очень бысто

In [15]:
minimum_rating = min(df_final['rating'].values)
 
maximum_rating = max(df_final['rating'].values)
 
print(f'Minimum/maximum оценки: {minimum_rating}/{maximum_rating}')

Minimum/maximum оценки: 0/1


In [16]:
reader = Reader(rating_scale=(minimum_rating,maximum_rating))
 
data = Dataset.load_from_df(df_final[['cookie_id', 'vacancy_id_', 'rating']], reader)

Предваритльное исследование
---

In [17]:
from surprise.model_selection import GridSearchCV

In [18]:
trainset = data.build_full_trainset()

In [20]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.33041003799507734
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

Полный цикл обучения

In [18]:
from surprise.model_selection import cross_validate

In [21]:
algo = SVD()

In [20]:
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3260  0.3262  0.3257  0.3251  0.3256  0.3257  0.0004  
MAE (testset)     0.2137  0.2138  0.2135  0.2132  0.2132  0.2135  0.0002  
Fit time          83.82   85.58   102.43  87.24   92.08   90.23   6.69    
Test time         12.06   13.94   11.96   13.10   14.30   13.07   0.95    


{'test_rmse': array([0.32604366, 0.32615915, 0.32568247, 0.32512348, 0.32558022]),
 'test_mae': array([0.21374677, 0.21375188, 0.21348058, 0.21321688, 0.21323298]),
 'fit_time': (83.82247614860535,
  85.58103847503662,
  102.43146705627441,
  87.24134159088135,
  92.08268356323242),
 'test_time': (12.05755066871643,
  13.937960147857666,
  11.958301067352295,
  13.101463317871094,
  14.300082445144653)}

In [21]:
trainset = data.build_full_trainset()

In [22]:
# обучаем алгоритм
algo.fit(trainset)

### Для тестирования

---

In [24]:
# не работает
testset = trainset.build_anti_testset()

MemoryError: 

In [ ]:
def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
 

In [ ]:
def precision_recall_at_k(predictions, k=5, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
top_5 = get_top_n(predictions, n=5)
# Print the recommended items for each user
for uid, user_ratings in top_5.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
from surprise.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=0.01)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))

### Получаем ответ
---

In [23]:
df_unic = df_final.groupby(['cookie_id','vacancy_id_']) # Осталяем только уникальные сочетания иначе он пойдет по всем записям

In [24]:
def get_prediction_for_list(user_list, model = algo, set_n = 5):
    top_n = {}
    
    for user_id in user_list:
        user_items = df_final[df_final['cookie_id'] == user_id]['vacancy_id_'].unique()  # взаимодействия пользователя
        candidates = df_final[~df_final['cookie_id'].isin(user_items)]['vacancy_id_'].unique()  # вакансии, с которыми пользователь не взаимодействовал
        testset = [[user_id, item_id, 1] for item_id in candidates]  # создание тестовой выборки
        predictions = model.test(testset)
        top_n[user_id] = sorted(predictions, key=lambda x: x.est, reverse=True)[:set_n]  # получение топ-5 рекомендаций

    # формирование результата в виде датафрейма
    result = pd.DataFrame(columns=['cookie_id', 'top_recommendations'])
    for user_id, predictions in top_n.items():
        item_list = [pred.iid for pred in predictions]
        result = result.append({'cookie_id': user_id, 'top_recommendations': item_list}, ignore_index=True)

    result.head()
    return result
    

In [45]:
def get_prediction_for_list2(user_list, model = algo, set_n = 5):
    top_n = {}
    
    for user_id in user_list:
        user_items = df_final[df_final['cookie_id'] == user_id]['vacancy_id_'].unique()  # взаимодействия пользователя
        candidates = df_final[~df_final['cookie_id'].isin(user_items)]['vacancy_id_'].unique()  # вакансии, с которыми пользователь не взаимодействовал
        testset = [[user_id, item_id, 1] for item_id in candidates]  # создание тестовой выборки
        predictions = model.test(testset)
        top_n[user_id] = sorted(predictions, key=lambda x: x.est, reverse=True)[:set_n]  # получение топ-5 рекомендаций

    # формирование результата в виде датафрейма
    result = pd.DataFrame(columns=['cookie_id', 'top_recommendations'])
    for user_id, predictions in top_n.items():
        item_list = [pred.iid for pred in predictions]
        
        result = pd.concat([result, {'cookie_id': user_id, 'top_recommendations': item_list}], ignore_index = True)

    result.head()
    return result

In [27]:
test = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')

In [28]:
test_list = list(test['cookie_id'])
print(test_list[:5])

['000cd76cd33f43d4a1ac1d16d10f8bf7', '0034bc7f404341ba8412665453e7825a', '00a6c5a64a274c55a836402bdeb3b2c4', '015937a125b14e74bdff1cddc49f9172', '01de50c280794cec8804f16f45f847b7']


In [47]:
# получение рекомендаций для списка пользователей только верхние
user_list = ['000cd76cd33f43d4a1ac1d16d10f8bf7',	'0034bc7f404341ba8412665453e7825a',
                 	'00a6c5a64a274c55a836402bdeb3b2c4', '015937a125b14e74bdff1cddc49f9172', 
                    '01de50c280794cec8804f16f45f847b7']

In [48]:
test1 = get_prediction_for_list(user_list)

C:\Users\pavel\AppData\Local\Temp\ipykernel_8176\3802414354.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'cookie_id': user_id, 'top_recommendations': item_list}, ignore_index=True)
C:\Users\pavel\AppData\Local\Temp\ipykernel_8176\3802414354.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'cookie_id': user_id, 'top_recommendations': item_list}, ignore_index=True)
C:\Users\pavel\AppData\Local\Temp\ipykernel_8176\3802414354.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'cookie_id': user_id, 'top_recommendations': item_list}, ignore_index=True)
C:\Users\pavel\AppData\Local\Temp\ipykernel_8176\3802414354.py:15: FutureWarning: The frame.ap

In [49]:
test1.head()

,cookie_id,top_recommendations
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[191243, 227171, 157801, 239245, 233914]"
1,0034bc7f404341ba8412665453e7825a,"[254064, 115977, 192614, 168669, 136804]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[186523, 203966, 180473, 229590, 162867]"
3,015937a125b14e74bdff1cddc49f9172,"[189127, 220465, 182069, 233914, 186650]"
4,01de50c280794cec8804f16f45f847b7,"[201216, 184067, 230397, 157311, 213741]"


In [ ]:
test2 = get_prediction_for_list2(user_list)

In [34]:
test.head()

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


In [ ]:
test_prediction = get_prediction_for_list(test_list)

In [ ]:
test_prediction2 = get_prediction_for_list2(test_list)

In [30]:
test_prediction.head(10)

,cookie_id,top_recommendations
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[191243, 227171, 157801, 239245, 233914]"
1,0034bc7f404341ba8412665453e7825a,"[254064, 115977, 192614, 168669, 136804]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[186523, 203966, 180473, 229590, 162867]"
3,015937a125b14e74bdff1cddc49f9172,"[189127, 220465, 182069, 233914, 186650]"
4,01de50c280794cec8804f16f45f847b7,"[201216, 184067, 230397, 157311, 213741]"
5,020c7d1d9a44451c9ef107fc2fe726f5,"[208194, 167622, 226277, 227526, 180473]"
6,02411265feae492ebca615b5b03e8bba,"[201679, 172620, 207098, 239245, 253980]"
7,028b4b7fd44b438b8b2b6198a74a8c88,"[219948, 201635, 163895, 239245, 172043]"
8,029c6b8042064d0899fd73f3290da565,"[112427, 227262, 204095, 233914, 117718]"
9,02db6a09b03f4459b77f0fa0acce6b42,"[247679, 108352, 106941, 244340, 153723]"


In [ ]:
work_data[(work_data['cookie_id'] == '02411265feae492ebca615b5b03e8bba') & (work_data['vacancy_id_'] == 201679)]

In [33]:
test_prediction.to_parquet('result.parquet')

In [ ]:
def precision_n(predictions,df_test_list_top, k=5):
  #recommendations num:
    n_rec_k = len(df_test_list_top)*k
  #true reccomendations:
    matches=0
    for row in range(len(predictions)):
      row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vacancy_id_'][row]))  
      matches += len(row_match)
  #Results:
    precisions = matches / n_rec_k if n_rec_k != 0 else 0
    return precisions

In [37]:
precision_n(test_prediction, test)

0.00025906735751295336

### Сохранение и загрузка модели

---

In [77]:
## SAVING TRAINED MODEL
from surprise import dump
import os
model_filename = "./modelSVD.pickle"
print (">> Starting dump")
# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=algo)
print (">> Dump done")
print(model_filename)

>> Starting dump
>> Dump done
./modelSVD.pickle


In [ ]:
## LOAD SAVED MODEL
def load_model(model_filename):
    print (">> Loading dump")
    from surprise import dump
    import os
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    print (">> Loaded dump")
    return loaded_model